## Attribution
This project incorporates code from the "QR Code Conditioned ControlNet Models for Stable Diffusion 1.5" by Dion Timmer, available on Hugging Face. The original code can be accessed at this [link](https://huggingface.co/DionTimmer/controlnet_qrcode-control_v1p_sd15).  
I'm thankful to Dion Timmer and contributors for their work and for making their resources available to the community. The use of the code from this repository is in accordance with the terms and conditions laid out by its license.

In [ ]:
import torch
from PIL import Image
from diffusers import StableDiffusionControlNetImg2ImgPipeline, ControlNetModel, DDIMScheduler
from diffusers.utils import load_image

controlnet = ControlNetModel.from_pretrained("DionTimmer/controlnet_qrcode-control_v1p_sd15",
                                             torch_dtype=torch.float16)

pipe = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16
)

pipe.enable_xformers_memory_efficient_attention()
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

def resize_for_condition_image(input_image: Image, resolution: int):
    input_image = input_image.convert("RGB")
    W, H = input_image.size
    k = float(resolution) / min(H, W)
    H *= k
    W *= k
    H = int(round(H / 64.0)) * 64
    W = int(round(W / 64.0)) * 64
    img = input_image.resize((W, H), resample=Image.LANCZOS)
    return img


# play with guidance_scale, controlnet_conditioning_scale and strength to make a valid QR Code Image

# qr code image
source_image = load_image("qrcode.png")
# initial image, anything
init_image = load_image("https://previews.123rf.com/images/chuyu/chuyu1212/chuyu121200092/17048157-blue-circuit-board-background-of-computer-motherboard.jpg")
condition_image = resize_for_condition_image(source_image, 768)
init_image = resize_for_condition_image(init_image, 768)
generator = torch.manual_seed(123121231)
image = pipe(prompt="beatiful, neon, 4k, cyberpunk",
             negative_prompt="ugly, disfigured, low quality, blurry, nsfw", 
             image=init_image,
             control_image=condition_image,
             width=768,
             height=768,
             guidance_scale=30,
             controlnet_conditioning_scale=3.0,
             generator=generator,
             strength=0.5, 
             num_inference_steps=150,
            )

image.images[0]

In [ ]:
image.images[0].save("ai_qrcode.png")